Se dispone de varios archivos raster en formato ".tif", donde cada uno almacena el pronóstico de precipitación acumulada para un período estacional y año específico en todo el dominio geográfico de Guatemala; estas predicciones fueron obtenidas a partir del modelo NextGEN.

Para cada uno de los archivos ".tif" se realiza lo siguiente:

01. Se toma el valor asociado a la ubicación geográfica de cada una de las estaciones meteorológicas.
02. Para cada estación meteorológica, se generan la siguiente variable objetivo:
    - Clasificación de la precipitación acumulada en cotas percentiles
03. Luego, se agrega a dataframe el nombre de la estación, el período analizado (Ej. ASO), el año, la latitud y longitud de la estación, así como la variable objetivo pronosticadas por NextGEN

Al finalizar el proces, el dataframe es exportado a un archivo en formato ".xlsx"

# IMPORTACIÓN DE DATOS Y LIBRERÍAS

In [ ]:
import pandas as pd
import rasterio
from rasterio.sample import sample_gen
import numpy as np
import os
from datetime import datetime

In [ ]:
# Cargar datos desde Excel
estaciones_df = pd.read_excel('Datos_estaciones.xlsx')  # Archivo con latitud y longitud de cada estación
climatologia_df = pd.read_excel('Climatologia_por_Periodos_Estacionales.xlsx')  # Archivo con climatología (media y percentiles de prec acum para cada perído estacional en cada estación meteorológica)

# DEFINICIÓN DE FUNCIONES

In [ ]:
# Función para extraer la precipitación de un archivo .tiff en todas las estaciones

def obtener_precAcum_en_estaciones(ruta_tiff, latitudes, longitudes):
    coords = [(lon, lat) for lon, lat in zip(longitudes, latitudes)]
    with rasterio.open(ruta_tiff) as src:
        precipitaciones = list(src.sample(coords))
        precipitaciones = [x[0] for x in precipitaciones]  # Obtenemos solo la precipitación
    return precipitaciones

In [ ]:
# Función para clasificar por percentiles

def clasificar_percentiles(precAcum, p1_3, p2_3):
    if p1_3 == 0 and p2_3 == 0:
        return 0
    else:
        if precAcum <= p1_3:
            return -1
        elif precAcum >= p2_3:
            return 1
        else:
            return 0

# PROCESAMIENTO DE ARCHIVOS .TIFF

In [ ]:
# Procesar todos los archivos .tiff
resultados = []
ruta_tiffs = 'Archivos_tiff/'  # Ruta donde están almacenados los archivos .tiff

In [ ]:
# Ciclo principal de ejecución:

for archivo_tiff in os.listdir(ruta_tiffs):
    if archivo_tiff.endswith('.tif'):
        
        # Extraer el período y año del nombre del archivo (asumiendo formato [periodo]-[año].tiff)
        Nombre_archivo = archivo_tiff.split('.')[0]
        periodo = Nombre_archivo.split('-')[0]
        año = Nombre_archivo.split('-')[1]
        
        # Extraer precipitación en las coordenadas de cada estación
        latitudes = estaciones_df['Latitud'].values
        longitudes = estaciones_df['Longitud'].values
        precipitacionesAcum = obtener_precAcum_en_estaciones(os.path.join(ruta_tiffs, archivo_tiff), latitudes, longitudes)
        
        for i, estacion in estaciones_df.iterrows():
            nombre_estacion = estacion['ESTACIÓN']
            latitud = estacion['Latitud']
            longitud = estacion['Longitud']
            
            # Obtener los datos de climatología para la fecha y estación
            clima = climatologia_df[(climatologia_df['ESTACIÓN'] == nombre_estacion) & (climatologia_df['Periodo_estacional'] == periodo)]
            media_clim = clima['Media'].values[0]
            p1_3 = clima['Percentil_1_3'].values[0]
            p2_3 = clima['Percentil_2_3'].values[0]
            
            # Precipitación predicha
            precipAcum_predicha = precipitacionesAcum[i]
            
            # Clasificación por percentiles
            percentil_clase = clasificar_percentiles(precipAcum_predicha, p1_3, p2_3)
            
            # Guardar resultados
            resultados.append([nombre_estacion, periodo, año, latitud, longitud, percentil_clase])

# Crear DataFrame final

In [ ]:
# Crear DataFrame final
resultados_df = pd.DataFrame(resultados, columns=['Estacion', 'Periodo', 'Año', 'Latitud', 'Longitud', 'percentil_NextGEN'])

# Exportar a Excel
resultados_df.to_excel('PronósticosNextGen.xlsx', index=False)